In [4]:
# Thuc hien lay cac url tu cac bai viet tren trang CNN
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from config import CHROME_DRIVER_PATH

FILE_PATH = os.path.join(os.getcwd(), "cnn_links.txt")

def write_links_to_file(links, file_path):
    with open(file_path, 'a+', encoding='utf-8') as file:
        for link in links:
            if not link.startswith("http"):
                file.write(f"https://edition.cnn.com{link}\n")

def scrape_cnn_articles():
    # Cấu hình ChromeDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Chạy Chrome ở chế độ headless
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # Đường dẫn đến ChromeDriver
    service = Service(CHROME_DRIVER_PATH)  # Thay thế đường dẫn đến chromedriver của bạn

    # Tạo đối tượng trình duyệt
    driver = webdriver.Chrome(service=service, options=chrome_options)

    categories = [
        '', 'health', 'us', 'world', 'politics', 'business', 'opinion', 
        'entertainment', 'style', 'travel', 'sports', 'climate', 'weather'
    ]
    links = set()

    for category in categories:
        driver.get(f"https://edition.cnn.com/{category}")
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-open-link]"))
            )
            elements = driver.find_elements(By.CSS_SELECTOR, "div[data-open-link]")
            for element in elements:
                temp = element.get_attribute("data-open-link")
                if temp and "video" not in temp:
                    links.add(temp)
        except Exception as e:
            print(f"An error occurred: {e}")

    write_links_to_file(links, FILE_PATH)
    driver.quit()

if __name__ == "__main__":
    scrape_cnn_articles()

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from config import CHROME_DRIVER_PATH

# Cấu hình ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy Chrome ở chế độ headless
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Đường dẫn đến ChromeDriver
service = Service(CHROME_DRIVER_PATH)  # Thay thế đường dẫn đến chromedriver của bạn

# Tạo đối tượng trình duyệt
driver = webdriver.Chrome(service=service, options=chrome_options)

# Đọc từ file ra danh sách các urls
with open('cnn_links.txt', 'r') as f:
    urls = [line.strip() for line in f.readlines()]

print(f"Have {len(urls)} urls")

new_urls = urls.copy()
invalid_urls = []

try:
    for url in new_urls:
        try:
            driver.get(url)
            # Tìm tất cả các thẻ a có class phù hợp
            elements = driver.find_elements(By.CLASS_NAME, "card.container__item.container__item--type-media-image.container__item--type-.container_list-headlines-with-read-times__item.container_list-headlines-with-read-times__item--type-")
            # In ra giá trị của thuộc tính href cho mỗi thẻ a tìm được
            for element in elements:
                href = element.get_attribute('data-open-link')
                new_urls.append(href)
        except Exception as e:
            print(f"Error accessing {url}")
            invalid_urls.append(url)
except KeyboardInterrupt:
    print("Process interrupted by user.")
finally:
    print(f"Have new {len(new_urls)} URLs")
    with open('new_urls.txt', 'a+') as f:
        for url in new_urls:
            f.write(url + '\n')

    print(f"Have {len(invalid_urls)} invalid URLs")
    with open('invalid_urls.txt', 'a+') as f:
        for url in invalid_urls:
            f.write(url + '\n')

    driver.quit()


Have 36 urls
Process interrupted by user.
Have new 381 URLs
Have 0 invalid URLs


In [1]:
with open('new_urls.txt', 'r') as f:
    urls = f.readlines()
    print(f"Have {len(urls)}")
    _urls = set(urls)
    fixed_urls = list(_urls)
    print(f"After, have {len(fixed_urls)}")
    with open('new_urls.txt', 'w') as f:
        for url in fixed_urls:
            f.write(url)

Have 2475
After, have 2102


In [2]:
import os
import hashlib
import requests
from bs4 import BeautifulSoup

def Hashhex(s):
    """Returns a heximal formated SHA1 hash of the input string.

    Args:
        s: The string to hash.

    Returns:
        A heximal formatted hash of the input string.
    """
    h = hashlib.sha1()
    h.update(s.encode())
    return h.hexdigest()

# Đọc các URL từ file
with open('new_urls.txt', 'r') as f:
    urls = f.readlines()

# Tạo thư mục download nếu chưa tồn tại
if not os.path.exists('download'):
    os.makedirs('download')


error_urls = []
# Duyệt qua từng URL, tải nội dung và lưu vào thư mục download
for url in urls:
    url = url.strip()  # Loại bỏ các ký tự xuống dòng hoặc khoảng trắng thừa
    try:
        # Tải nội dung HTML
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "html.parser")

        # Lấy tiêu đề bài viết
        title_tag = soup.find('title')
        if title_tag:
            title = title_tag.get_text()
        else:
            title = url

        # Tạo tên file bằng cách sử dụng hàm Hashhex
        file_name = Hashhex(title) + ".html"

        # Đường dẫn file để lưu
        file_path = os.path.join('download', file_name)

        # Lưu nội dung HTML vào file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(str(soup.prettify()))
    except Exception as e:
        print(f"Không thể tải nội dung từ {url}.")
        error_urls.append(url)

Nội dung từ https://web.archive.org/web/20240711015833/https://www.cnn.com/2024/07/04/us/south-padre-island-shark-bites/index.html?iid=cnn_buildContentRecirc_end_recirc đã được lưu vào download\ed5a4626ac51527b26b8933ece5fd2d76bbf3aa8.html
Nội dung từ https://web.archive.org/web/20240711015808/https://www.cnn.com/2024/07/10/us/5-things-to-know-for-july-10-nato-summit-2024-race-beryl-aftermath-immigration-human-plague/index.html?iid=cnn_buildContentRecirc_end_recirc đã được lưu vào download\10ca16eb1a42b7b113e3e5705935eb649124a281.html
Nội dung từ https://web.archive.org/web/20240711015833/https://www.cnn.com/2024/07/04/us/south-padre-island-shark-bites/index.html?iid=cnn_buildContentRecirc_end_recirc đã được lưu vào download\ed5a4626ac51527b26b8933ece5fd2d76bbf3aa8.html
Nội dung từ https://web.archive.org/web/20240711015808/https://www.cnn.com/2024/07/10/us/5-things-to-know-for-july-10-nato-summit-2024-race-beryl-aftermath-immigration-human-plague/index.html?iid=cnn_buildContentRecirc_